In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_raw_data=pd.read_csv('../input/titanic/train.csv')
test_raw_data=pd.read_csv('../input/titanic/test.csv')

In [ ]:
trainrow=train_raw_data.shape[0]
testrow=test_raw_data.shape[0]
y_train=train_raw_data['Survived'].copy()
train_raw_data=train_raw_data.drop(['Survived'],1)

In [ ]:
combine=pd.concat([train_raw_data,test_raw_data])
combine.head()

In [ ]:
print(combine.shape)
print(train_raw_data.shape)
print(test_raw_data.shape)

In [ ]:
combine.isnull().sum()

In [ ]:
#here we can se only three features has missing data
#Emabarked lets replace it with most frequent data
combine['Embarked']=combine['Embarked'].fillna(combine['Embarked'].value_counts()[0])

In [ ]:
#here we can see cabin does not matter to Survive so lets delete it
combine=combine.drop(['Cabin'],1)

In [ ]:
combine.loc[combine['Fare'].isnull()]

In [ ]:
#fare depend on Pclass,Sex, and age
combine.loc[(combine['Pclass']==3) & (combine['Sex']=="male") & (combine['Age']<65) & (combine['Age']>55)]

In [ ]:
train_raw_data=train_raw_data.drop(['PassengerId','Ticket'],1)
passengerids=test_raw_data['PassengerId']
test_raw_data=test_raw_data.drop(['PassengerId','Ticket'],1)
passengerids

In [ ]:
train_raw_data.head()

In [ ]:
combine=[train_raw_data,test_raw_data]
for dataset in combine:
    dataset['familysize']=dataset['SibSp']+dataset['Parch']+1
train_raw_data.head()

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
train_raw_data.head()

In [ ]:
train_raw_data.Title.unique()

In [ ]:
test_raw_data.Title.unique()

In [ ]:
train_raw_data['Title'].value_counts()

In [ ]:
for dataset in combine:
    dataset['Title']=dataset['Title'].map({"Mr":0,"Miss":1,"Mrs":2,"Master":3,"Dr":3,"Rev":3,"Major":3,"Col":3,"Mlle":3,"Don":3,"Jonkheer":3,"Countess":3,"Sir":0,"Capt":3,"Mme":2,"Lady":1,"Ms":0,"Dona":3})
train_raw_data.head()

In [ ]:
train_raw_data=train_raw_data.drop(['Name'],1)
test_raw_data=test_raw_data.drop(['Name'],1)
combine=[train_raw_data,test_raw_data]
train_raw_data.head()

In [ ]:
train_raw_data=train_raw_data.drop(['SibSp','Parch'],1)
test_raw_data=test_raw_data.drop(['SibSp','Parch'],1)
combine=[train_raw_data,test_raw_data]
train_raw_data.head()

In [ ]:
for dataset in combine:
    dataset['Sex']=dataset['Sex'].map({'male':0,'female':1})
train_raw_data.head()

In [ ]:
train_raw_data.Pclass.unique()

In [ ]:
for dataset in combine:
    for i in range(0,2):
        for j in range(0,3):
            temp_dataset=dataset[(dataset['Sex']==i) &  (dataset['Pclass']==j+1)]['Age'].dropna()
            temp_dataset
            #print(str(temp_dataset.median())+"  "+str(i)+"  "+str(j+1))
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex==i) & (dataset.Pclass==j+1),'Age']=int(temp_dataset.median())

In [ ]:
test_raw_data.loc[test_raw_data.Fare.isnull(),'Fare']=test_raw_data[test_raw_data.Pclass==3]['Fare'].median()

In [ ]:
test_raw_data.isnull().sum()

In [ ]:
combine=[train_raw_data,test_raw_data]

In [ ]:
combine[0].head()

In [ ]:
combine[1].head()

In [ ]:
train_raw_data['Embarked'].unique()
    

In [ ]:
for dataset in combine:
    dataset['Embarked']=dataset['Embarked'].map({'S':0,'C':1,'Q':2})

In [ ]:
train_raw_data.head()

In [ ]:
x_train=train_raw_data.loc[:,'Pclass':'Title']
y_train=train_raw_data.loc[:,'Survived']
x_test=test_raw_data.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()
scaler.fit(x_train)
x_scaled_train=scaler.transform(x_train)
x_scaled_train

In [ ]:
x_scaled_test=scaler.transform(x_test)
x_scaled_test

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
reg=LogisticRegression()
reg.fit(x_scaled_train,y_train)

In [ ]:
reg.score(x_scaled_train,y_train)

In [ ]:
y_pred=reg.predict(x_scaled_test)
y_pred

In [ ]:
passengerids.head()

In [ ]:
submission = pd.DataFrame({
        "PassengerId": passengerids,
        "Survived": y_pred
    })
submission

In [ ]:
submission.to_csv('submission.csv', index=False)